In [1]:
import os
import shutil
import cv2
import random
import glob
from skimage.util import random_noise
import numpy as np
import shutil

In [2]:
# defining the function to ignore the files
# if present in any folder
def ignore_files(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir, f))]
 
# calling the shutil.copytree() method and
# passing the src,dst,and ignore parameter
shutil.copytree(r"S:\Logo2k+ for classification",
                r"S:\Logo2k+ for classification augmentedv2",
                ignore=ignore_files)

'S:\\Logo2k+ for classification augmentedv2'

In [3]:
def augment_pdf_to_mobile(image):
    # Add random noise to simulate inaccuracies
    noisy_image = random_noise(image, mode='gaussian', var=0.01)

    # Simulate shadows by darkening the entire image
    shadow_intensity = random.uniform(0.4, 0.9)  # Adjust the shadow intensity based on your requirements
    shadowed_image = noisy_image * shadow_intensity

    return shadowed_image

In [4]:
def apply_random_perspective(image):
        
    height, width = image.shape[:2]

    # Define the source and destination points for perspective transformation
    src_points = np.float32([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]])

    # Calculate the maximum displacement limit
    max_displacement = int(min(width, height) * 0.3)

    # Generate random positive displacements for each corner of the image
    displacement_range = np.random.randint(0, max_displacement + 1, size=(4, 2))

    dst_points = np.zeros((4, 2), dtype=np.float32)
    
    dst_points[0][0], dst_points[0][1] = src_points[0][0] + displacement_range[0][0], src_points[0][1] + displacement_range[0][1]
    dst_points[1][0], dst_points[1][1] = src_points[1][0] - displacement_range[1][0], src_points[1][1] + displacement_range[1][1]
    dst_points[2][0], dst_points[2][1] = src_points[2][0] - displacement_range[2][0], src_points[2][1] - displacement_range[2][1]
    dst_points[3][0], dst_points[3][1] = src_points[3][0] + displacement_range[3][0], src_points[3][1] - displacement_range[3][1]
    
    # Generate random perspective transformation matrix
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    
    #  white mask
    mask = cv2.warpPerspective(np.ones_like(image), perspective_matrix, (width, height))
    # Fill the areas outside the transformed region with white

    # Apply the perspective transformation to the image
    transformed_image = cv2.warpPerspective(image, perspective_matrix, (width, height))
    
    # add the white background
    transformed_image[np.where(mask == 0)] = 255
    
    # apply noise
    transformed_image = augment_pdf_to_mobile(transformed_image)
    
    # crop
    x_coordinates = dst_points[:, 0]
    y_coordinates = dst_points[:, 1]

    top_left = (np.min(y_coordinates), np.min(x_coordinates))
    bottom_right = (np.max(y_coordinates), np.max(x_coordinates))
    
    transformed_image = transformed_image[int(top_left[0]):int(bottom_right[0]), int(top_left[1]):int(bottom_right[1])]

    return transformed_image, dst_points

In [5]:
logo_path = r"C:\Users\Florian Moga\Downloads\paypal logo 2.png"
logo = cv2.imread(logo_path)
h, w, _ = logo.shape

In [8]:
shadow_logo = augment_pdf_to_mobile(logo)
cv2.imshow("shadowed", shadow_logo)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
perspective_logo, dst_points = apply_random_perspective(logo)
cv2.imshow("perspective", perspective_logo)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
ROOT, DST, DST_test = r"S:\Logo2k+ for classification\train", r"S:\Logo2k+ for classification augmentedv2\train", r"S:\Logo2k+ for classification augmentedv2\test"
NUM_SAMPLES = 20

In [12]:
for directory in os.listdir(ROOT):
    
    root_dir_path = os.path.join(ROOT, directory)
    dst_dir_path = os.path.join(DST, directory)
    dst_testdir_path = os.path.join(DST_test, directory)

    # choose a random logo from each folder
    class_files = os.listdir(root_dir_path)
    random_index = random.randint(0, len(class_files)-1)
    photo_path = class_files[random_index]
    
    # copy it to DST
    shutil.copy(os.path.join(root_dir_path, photo_path), os.path.join(dst_dir_path, photo_path))
    
    # augment it
    photo = cv2.imread(os.path.join(root_dir_path, photo_path))
    
    #train
    for i in range(NUM_SAMPLES//2):
        perspective_logo, _ = apply_random_perspective(photo)
        path = f"perspective{i}_"+photo_path
        full_path = os.path.join(dst_dir_path, path)
        cv2.imwrite(full_path, 255*perspective_logo)
        
        shadow_logo = augment_pdf_to_mobile(photo)
        path = f"shadowed{i}_"+photo_path
        full_path = os.path.join(dst_dir_path, path)
        cv2.imwrite(full_path, 255*shadow_logo)
        
    # test
    for i in range(NUM_SAMPLES//4):
        shadow_logo = augment_pdf_to_mobile(photo)
        path = f"shadowed{i}_"+photo_path
        full_path = os.path.join(dst_testdir_path, path)
        cv2.imwrite(full_path, 255*shadow_logo)
        
        perspective_logo, _ = apply_random_perspective(photo)
        path = f"perspective{i}_"+photo_path
        full_path = os.path.join(dst_testdir_path, path)
        cv2.imwrite(full_path, 255*perspective_logo)


In [21]:
random.randint(1, 2)

1

In [42]:
shutil.copytree(r"C:\Users\Florian Moga\Desktop\Code\classes for embedding similarity",
                r"C:\Users\Florian Moga\Desktop\Code\classes for embedding similarity augmented",
                ignore=ignore_files)

'C:\\Users\\Florian Moga\\Desktop\\Code\\classes for embedding similarity augmented'

In [44]:
ROOT, DST = r"C:\Users\Florian Moga\Desktop\Code\classes for embedding similarity\untrained", r"C:\Users\Florian Moga\Desktop\Code\classes for embedding similarity augmented\untrained"
NUM_SAMPLES = 12

In [45]:
for directory in os.listdir(ROOT):
    
    root_dir_path = os.path.join(ROOT, directory)
    dst_dir_path = os.path.join(DST, directory)

    # choose a random logo from each folder
    class_files = os.listdir(root_dir_path)
    random_index = random.randint(0, len(class_files)-1)
    photo_path = class_files[random_index]
    
    # copy it to DST
    shutil.copy(os.path.join(root_dir_path, photo_path), os.path.join(dst_dir_path, photo_path))
    
    # augment it
    photo = cv2.imread(os.path.join(root_dir_path, photo_path))
    
    #train
    for i in range(NUM_SAMPLES//2):
        perspective_logo, _ = apply_random_perspective(photo)
        path = f"perspective{i}_"+photo_path
        full_path = os.path.join(dst_dir_path, path)
        cv2.imwrite(full_path, 255*perspective_logo)
        
        shadow_logo = augment_pdf_to_mobile(photo)
        path = f"shadowed{i}_"+photo_path
        full_path = os.path.join(dst_dir_path, path)
        cv2.imwrite(full_path, 255*shadow_logo)

